<a href="https://colab.research.google.com/github/ricglz/CE888_activities/blob/main/assignment/Data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! [ ! -z "$COLAB_GPU" ] && pip install torch torchvision kaggleDownloader

# Preparations

----

In [ ]:
from kaggleDownloader import get_dataset
from torchvision.datasets import ImageFolder
from os import path
from tqdm import tqdm
from google.colab import drive

import torchvision.utils as t_utils
import torchvision.transforms as T

In [ ]:
drive_path = '/content/gdrive'
drive.mount(drive_path, force_remount=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
classes = ['Fire', 'No_Fire']
training_path = path.join(drive_path, 'MyDrive/Essex/Datasets/Flame/Training')

In [ ]:
def save_image(img, label, prefix):
    klass = classes[label]
    img_path = path.join(training_path, f'{klass}/{prefix}_{index}.png')
    t_utils.save_image(img, img_path)

In [ ]:
# To soft start the drive mount
ImageFolder(training_path)
print()

# Add new dataset to the existent one

----

One of the principal problems of the current dataset is that the images are basically a burst of shots of the same environment, this leads to the problem that the model learns to recognize the environment instead of recognizing fire

To avoid this we will use another [dataset that can be found in kaggle](https://www.kaggle.com/phylake1337/fire-dataset). Which may help the model to actually recognize the fire

In [ ]:
get_dataset('kaggle datasets download -d phylake1337/fire-dataset')


Download completed..
Unzipping the zip files

Zip Files unzipped

 Directory contains the following files :  ['.config', 'fire-dataset.zip', 'kaggle.json', 'gdrive', 'fire_dataset', 'sample_data']

Remove zip files ? (yes/no) :yes

 Directory contains the following files :  ['.config', 'kaggle.json', 'gdrive', 'fire_dataset', 'sample_data']


In [ ]:
resize = T.Resize((254, 254))
transforms = T.Compose([resize, T.ToTensor()])

In [ ]:
extra_data = ImageFolder('/content/fire_dataset', transforms)
len(extra_data)

999

## Save the image

In [ ]:
for index, (img, label) in enumerate(tqdm(extra_data)):
    save_image(img, label, 'extra_dataset')

100%|██████████| 999/999 [01:16<00:00, 13.13it/s]


# Augment the data

----

We'll augment the current dataset by
- Modifying the brightness of the image
- Doing random flips (vertical and horizontal)
- Doing random rotation

In [ ]:
# transforms = T.Compose([
#   resize,
#   T.ColorJitter(brightness=0.55),
#   T.RandomRotation(degrees=5),
#   T.ToTensor(),
# ])

SyntaxError: ignored

In [ ]:
# train_ds = ImageFolder(training_path, transforms)

In [ ]:
# for index, (img, label) in enumerate(tqdm(train_ds)):
#     save_image(img, label, 'augmented')

# Balance datasets

----

In [ ]:
transforms = T.Compose([
  resize,
  T.ColorJitter(brightness=0.25, contrast=0.25),
  T.RandomRotation(degrees=5),
  T.RandomHorizontalFlip(),
  T.RandomVerticalFlip(),
  T.ToTensor(),
])

train_ds = ImageFolder(training_path, transforms)

In [ ]:
import numpy as np

targets = np.array(train_ds.targets)

In [ ]:
fire_data_count = np.count_nonzero(targets == 0)
f'Fire data: {fire_data_count}'

'Fire data: 25829'

In [ ]:
non_fire_data_count = np.count_nonzero(targets == 1)
f'Non-Fire data: {non_fire_data_count}'

'Non-Fire data: 14632'

In [ ]:
klass_counts = [fire_data_count, non_fire_data_count]
minor_klass = np.argmin(klass_counts)
minor_count = min(klass_counts)
max_count = max(klass_counts)

In [ ]:
images_to_save = min(max_count - minor_count, minor_count)
images_to_save

11197

## Save the image

In [ ]:
saved_images = 0
for index, (img, label) in enumerate(tqdm(train_ds)):
    if label == minor_klass:
      save_image(img, label, 'balance')
      saved_images += 1
      if saved_images == images_to_save:
        break

 92%|█████████▏| 37025/40461 [3:24:15<20:12,  2.83it/s]

# Check final count

----

In [ ]:
train_ds = ImageFolder(training_path, transforms)
targets = np.array(train_ds.targets)

In [ ]:
fire_data_count = np.count_nonzero(targets == 0)
f'Fire data: {fire_data_count}'

'Fire data: 25829'

In [ ]:
non_fire_data_count = np.count_nonzero(targets == 1)
f'Non-Fire data: {non_fire_data_count}'

'Non-Fire data: 25829'

In [ ]:
f'Total: {fire_data_count + non_fire_data_count}'

'Total: 51658'